# 💬 Claimify reproduction study

In [1]:
# autoreload imports
%load_ext autoreload
%autoreload 2

In [2]:
from utils.chat import load_md_prompts

PROMPT_TEMPLATES = load_md_prompts("claimify")

## 1. Input Question and Answer

In [13]:
%pip install charboundary[numpy,onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.3 MB/s  0:00:0010.4 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 12.0 MB/s  0:00:01 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 55.7 MB/s  0:00:00 52.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [charboundary]m━━━━ 8/9 [charboundary]
Note: you may need to restart the kernel to use updated packages.


In [27]:
# basic chatprompttemplate llm chat with gpt-5 using 'selection' prompt template
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

import ssl
import certifi

chat = ChatOpenAI(model="gpt-5")
excerpt = """"""
question = """"""
sentence = """"""
chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt=excerpt, question=question, sentence=sentence))

AIMessage(content='Sentence: \n<>\n\n4-step stream of consciousness thought process (1. reflect on criteria at a high-level -> 2. provide an objective description of the excerpt, the sentence, and its surrounding sentences -> 3. consider all possible perspectives on whether the sentence explicitly or implicitly contains a specific and verifiable proposition, or if it just contains an introduction for the following \nsentence(s), a conclusion for the preceding sentence(s), broad or generic statements, opinions, interpretations, speculations, statements about a lack of information, etc. -> 4. only if it contains a specific and verifiable proposition: reflect on whether any changes are needed to ensure that the entire sentence only contains verifiable information):\n1) A sentence must assert at least one checkable claim to qualify; absence of content cannot contain a proposition. 2) The provided "Sentence" field is empty, and no surrounding context is given. 3) An empty sentence cannot ex

In [ ]:
from utils.claimify import SelectionResponse

In [8]:
chat = ChatOllama(model="qwen3:14b")
output = chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt="todo", question="todo", sentence="todo"))

In [12]:
output.content

'<think>\nOkay, let\'s tackle this problem step by step. The user wants me to determine if a given sentence from a response contains a specific and verifiable proposition based on the provided rules.\n\nFirst, I need to recall the criteria. A specific and verifiable proposition is something that can be checked for truth, like a fact. The rules mention that if the sentence is about a lack of information, it doesn\'t count. Also, it doesn\'t matter if the proposition is true or false, just whether it\'s specific and verifiable. Ambiguous terms are okay as long as the fact-checker can resolve them.\n\nNow, looking at the example given in the problem statement. The user provided a scenario where the sentence is "John" following the question "Who is the CEO of Company X?" and the preceding sentence. The answer here is that "John" contains a specific proposition because it directly answers the question with a name, which can be verified.\n\nAnother example is when the preceding sentence ment

In [24]:
from typing import List
import nltk

def split_into_sentences(text: str) -> List[str]:
    """
    Splits a block of text into sentences, handling paragraphs and lists.
    This replicates the methodology from Appendix C.1 of the Claimify paper.
    
    Args:
        text: The input text to split
        
    Returns:
        A list of sentence strings
    """
    
    sentences = []
    # First, split by newlines to handle paragraphs and list items
    paragraphs = text.split('\n')
    for para in paragraphs:
        if para.strip():  # Avoid empty paragraphs
            # Then, use NLTK's sentence tokenizer on each paragraph
            sentences.extend(nltk.sent_tokenize(para))
    return sentences

split_into_sentences("""
List The Wall Street Journal předtím uvedl, že Evropané na schůzi představili vlastní plán jednání o ukončení války. Podle dokumentu před přiznáním jakýchkoli ústupků Rusku je nutné vyhlásit příměří. Eventuální územní ústupky musí být reciproční, musí je tedy udělat Ukrajina i Rusko. Plán také žádá silné bezpečnostní záruky pro Ukrajinu včetně členství v NATO. Trump v minulosti dával najevo, že takový postup si nepřeje.

Bílý dům zvažuje pozvání Zelenského na Aljašku
V pátek 15. srpna se Trump sejde na Aljašce se svým ruským protějškem Vladimirem Putinem. Půjde o dvoustrannou schůzku, Bílý dům však podle informací stanice NBC News stále zvažuje, že na Aljašku pozve také ukrajinského prezidenta Volodymyra Zelenského.

NBC News se odvolává na vysoce postaveného amerického činitele a tři osoby, které byly o interních diskusích v Bílém domě informovány.

„Diskutuje se o tom,“ citovala stanice jednoho z lidí seznámeného s obsahem jednání. Vysoce postavený americký činitel a lidé obeznámení s rozhovory zároveň uvedli, že návštěva Zelenského zatím nebyla s konečnou platností dohodnuta a není jasné, zda se Zelenskyj nakonec jednání na Aljašce zúčastní.

Lipavský o schůzce Trumpa s Putinem: Hranice Ukrajiny neposune vydírání

Vysoce postavený představitel Trumpovy administrativy k tomu řekl, že je to „rozhodně“ možné. „Všichni silně doufají, že k tomu dojde,“ dodal.

Trump podle něj zůstává otevřen možnosti trojstranného summitu s oběma lídry. Momentálně se ale Bílý dům „soustředí na přípravu bilaterální schůzky, o kterou požádal prezident Putin“, řekl činitel.

Ukrajinský prezident Volodymyr Zelenskyj v sobotu varoval, že jakékoliv řešení rusko-ukrajinského konfliktu učiněné bez účasti Ukrajiny bude řešením proti míru. Dodal, že Ukrajinci nevydají svou zemi okupantům.

Šéf Kremlu tento týden prohlásil, že není proti setkání se Zelenským, ale pro takovou schůzku podle něj ještě nejsou podmínky. Zelenskyj opakovaně vyzval k osobnímu jednání mezi ním a Putinem s cílem ukončit ruskou invazi na Ukrajinu.

Ukrajina se brání rozsáhlé ruské vojenské agresi od února 2022. Zatím není jasné, jestli je Moskva připravena vzdát se jakéhokoliv území, které nyní okupuje. Kyjev dosud podle agentury Bloomberg dával najevo, že ruskou okupaci a anexi svého území nepřijme.

Podle agentury Reuters sice Ukrajina signalizovala flexibilitu při hledání cest k ukončení války, ale přijmout ztrátu zhruba pětiny území by bylo velmi bolestivé a politicky složité pro Zelenského a jeho vládu.
)
Zdroj: https://www.idnes.cz/zpravy/zahranicni/zelensky-trump-putin-aljaska-schuzka-usa-rusko-ukrajina-valka.A250810_071424_zahranicni_svm""")

['List The Wall Street Journal předtím uvedl, že Evropané na schůzi představili vlastní plán jednání o ukončení války.',
 'Podle dokumentu před přiznáním jakýchkoli ústupků Rusku je nutné vyhlásit příměří.',
 'Eventuální územní ústupky musí být reciproční, musí je tedy udělat Ukrajina i Rusko.',
 'Plán také žádá silné bezpečnostní záruky pro Ukrajinu včetně členství v NATO.',
 'Trump v minulosti dával najevo, že takový postup si nepřeje.',
 'Bílý dům zvažuje pozvání Zelenského na Aljašku',
 'V pátek 15. srpna se Trump sejde na Aljašce se svým ruským protějškem Vladimirem Putinem.',
 'Půjde o dvoustrannou schůzku, Bílý dům však podle informací stanice NBC News stále zvažuje, že na Aljašku pozve také ukrajinského prezidenta Volodymyra Zelenského.',
 'NBC News se odvolává na vysoce postaveného amerického činitele a tři osoby, které byly o interních diskusích v Bílém domě informovány.',
 '„Diskutuje se o tom,“ citovala stanice jednoho z lidí seznámeného s obsahem jednání.',
 'Vysoce postav

In [42]:
from utils.claimify import ClaimifyPipeline
from utils.chat import chat_factory

pipeline = ClaimifyPipeline(chat_factory("gpt-5-mini"))

2025-08-10 12:00:50,320 - claimify.pipeline - INFO - Structured outputs enabled for improved reliability


In [43]:
article = """European allies have rallied behind Ukraine in a renewed surge of support, insisting that any peace talks with Russia must include Kyiv.

A joint statement issued by the leaders of the UK, France, Italy, Germany, Poland, Finland and the European Commission came ahead of US President Donald Trump's meeting with his Russian counterpart Vladimir Putin in Alaska on Friday.

A White House official has said that Trump is willing to hold a trilateral meeting which would also include Ukrainian President Volodymyr Zelensky, but, for now, it remains a Trump-Putin summit, as initially requested by the Russian leader.

Zelensky has said any agreements without Kyiv will amount to "dead decisions".

Trump has previously suggested that he could start by meeting only with Putin, telling reporters he planned to "start off with Russia." But the US president also said that he believed "we have a shot at" organising a trilateral meeting with both Putin and Zelensky.

Whether Putin would agree to this is unclear - he has refused several opportunities to hold direct talks, and the two leaders have not met face-to-face since Putin launched a full-scale invasion of Ukraine more than three years ago.

Speaking on Friday, Trump also suggested that there "will be some swapping of territories" in order for Moscow and Kyiv to reach an agreement - to which Zelensky reacted strongly.

"We will not reward Russia for what it has perpetrated," he said on Telegram. "Any decisions against us, any decisions without Ukraine, are also decisions against peace."

"The Russians... still impose the idea of 'exchanging' Ukrainian territory for Ukrainian territory, with consequences that guarantee nothing but more convenient positions for the Russians to resume the war," he added defiantly.

CBS, the BBC's US media partner, has reported that the White House is trying to sway European allies to accept an agreement that would include Russia taking the entire Donbas region in eastern Ukraine, and keeping the Crimean Peninsula.

The European leaders, in their statement released late on Saturday night, stressed that "international borders must not be changed by force".

"Ukraine has the freedom of choice over its own destiny," they said, stressing that their nations would continue to support Ukraine diplomatically, militarily and financially.

More on the war in Ukraine
Ukraine in maps: Tracking the war with Russia
Russian attacks on Ukraine double since Trump inauguration
On Ukraine's front line, twisted wreckage shows sanctions haven't yet stopped Russia
The leaders also said that a "diplomatic solution" is critical, not just to protect Ukraine - but also Europe's security.

It's not just Ukraine that is struggling to be part of the Alaska meeting.

European allies are also worried about their lack of influence over the outcome of any agreement that Trump could reach with Putin.

In a post on X on Saturday, French President Emmanuel Macron raised concerns about Russia and the US excluding European involvement.

"Europeans will also necessarily be part of the solution, as their own security is at stake," he wrote.

On Sunday, Zelensky thanked the allies for their support.

"The end of the war must be fair, and I am grateful to everyone who stands with Ukraine and our people today for the sake of peace in Ukraine, which is defending the vital security interests of our European nations," he said.

Europe has taken a tough approach to Moscow - including imposing sanctions against Russian entities and providing military aid for Ukraine.

Zelensky said he told Macron in a phone call on Saturday that the key was to make sure "the Russians do not get to deceive anyone again".

"We all need a genuine end to the war and reliable security foundations for Ukraine and other European nations," the Ukrainian leader said.

UK government handout From left to right: Secretary of the National Security and Defence Council of Ukraine, UK Foreign Secretary David Lammy, US Vice-President JD Vance and Andriy Yermak, Zelensky's chief of staffUK government handout
David Lammy and JD Vance hosted Zelensky's top aides on Saturday
US diplomacy with Europe and Ukraine fell to Vice-President JD Vance on Saturday, when he visited the UK and held talks with Foreign Secretary David Lammy as well as two of Zelensky's top aides.

Thanking Vance for the discussions, Andriy Yermak, the head of Zelensky's office, stressed the need for Ukraine to be included.

"A reliable, lasting peace is only possible with Ukraine at the negotiating table," he said. "A ceasefire is necessary - but the frontline is not a border."

The summit in Alaska, the territory which Russia sold to the US in 1867, would be the first between sitting US and Russian presidents, since Joe Biden met Putin in Geneva in June 2021.

Nine months later, Moscow sent troops into Ukraine.

In 2022, the Kremlin announced the annexation of four Ukrainian regions - Donetsk, Lugansk, Zaporizhzhia and Kherson - despite not having full control over them.

Moscow has failed to achieve a decisive breakthrough in its full-scale invasion, but occupies large swathes of Ukraine's eastern territory. Ukrainian offensives, meanwhile, have not been able to push the Russian forces back."""

In [44]:
pipeline.run(article)

2025-08-10 12:00:56,157 - claimify.pipeline - INFO - Processing 39 sentences
2025-08-10 12:00:56,158 - claimify.pipeline - INFO - Processing sentence 1/39: European allies have rallied behind Ukraine in a renewed surge of support, insisting that any peace ...
2025-08-10 12:01:05,038 - claimify.pipeline - INFO - SELECTION: Sentence verifiable


AttributeError: 'NoneType' object has no attribute 'format_messages'

In [41]:
from utils.chat import get_prompt
prompt = get_prompt("claimify", "decomposition", {"question": question, "excerpt": excerpt, "sentence": sentence})
# type of prompt
prompt

[SystemMessage(content='You are an assistant for a group of fact-checkers. You will be given a question, which was asked about a source text (it may be referred to by other names, \ne.g., a dataset). You will also be given an excerpt from a response to the question. If it contains "[...]", this means that you are NOT seeing all sentences in the response. You will also be given a particular sentence from the response. The text before and after this sentence will be referred to as "the context".\n\nYour task is to identify all specific and verifiable propositions in the sentence and ensure that each proposition is decontextualized. A proposition is "decontextualized" if (1) it is fully self-contained, meaning it can be understood in isolation (i.e., without the question, the context, and the other propositions), AND (2) its meaning in isolation matches its meaning when interpreted alongside the question, the context, and the other propositions. The propositions should also be the simples

In [49]:
get_prompt("claimify", "selection", {"question": question, "excerpt": excerpt, "sentence": sentence})

[SystemMessage(content='You are an assistant to a fact-checker. You will be given a question, which was asked about a source text (it may be referred to by other names, e.g., a \ndataset). You will also be given an excerpt from a response to the question. If it contains "[...]", this means that you are NOT seeing all sentences in the response. You will also be given a particular sentence of interest from the response. Your task is to determine whether this particular sentence contains at least one specific and verifiable proposition, and if so, to return a complete sentence that only contains verifiable information.   \n\nNote the following rules:\n- If the sentence is about a lack of information, e.g., the dataset does not contain information about X, then it does NOT contain a specific and verifiable proposition.\n- It does NOT matter whether the proposition is true or false.\n- It does NOT matter whether the proposition is relevant to the question.\n- It does NOT matter whether the 